In [1]:
# File location and type
file_location1 = "/FileStore/tables/table1.csv"
file_location2 = "/FileStore/tables/table2.csv"
file_type = "csv"

# CSV options
infer_schema = "false"
first_row_is_header = "true"
delimiter = ","

# The applied options are for CSV files. For other file types, these will be ignored.
df1 = spark.read.format(file_type) \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .load(file_location1)

df2 = spark.read.format(file_type) \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .load(file_location2)

temp_table_name = "table1"
df1.createOrReplaceTempView(temp_table_name)
temp_table_name2 = "table2"
df1.createOrReplaceTempView(temp_table_name2)

In [2]:
# below cells meaning:

Master_without_dublicate1 (M1): table have data from master and have rows which is single occuring.
OnlyDuplicateFromMaster1 (D) : table have data from master and have rows which is occuring ultiple time.

In [3]:
Master_without_dublicate1 = spark.sql( "select distinct a.* from table1 a inner join (select apid,address_type ,count(*) from table1 group by apid,address_type having count(*)=1) b on a.apid=b.apid and a.address_type=b.address_type where a.apid in ('366','896','970')");
                                      
Master_without_dublicate1.createOrReplaceTempView("Master_without_dublicate")                         
         
OnlyDuplicateFromMaster1 = spark.sql( "select a1.apid,a1.address_type,a1.address_line_1,a1.address_line_2,a1.city,a1.state, a1.postal_code,a1.Country_code, a1.validated_by ,a1.creation_timestamp ,max(a1.last_update_timestamp) last_update_timestamp from (select distinct a.* from table1 a inner join (select apid,address_type,count(*) from table1 group by apid,address_type having count(*)>1) b on a.apid=b.apid and a.address_type=b.address_type ) a1  where a1.apid in ('366','896','970') group by apid,address_type,address_line_1,address_line_2,city,state, postal_code,Country_code, validated_by,creation_timestamp");                 OnlyDuplicateFromMaster1.createOrReplaceTempView("OnlyDuplicateFromMaster")  



In [4]:
%sql
select * from OnlyDuplicateFromMaster

apid,address_type,address_line_1,address_line_2,city,state,postal_code,Country_code,validated_by,creation_timestamp,last_update_timestamp
896,Mailing,5711 nw 48 lane,null,lauderhill,VA,9537,USA,False,2019-11-24T03:23:28.000+0000,2019-11-24T15:23:33.000+0000
366,Mailing,4217 si streat,null,riverbank,AR,9537,USA,False,2019-11-24T03:10:06.000+0000,2019-11-30T03:10:12.000+0000
970,Mailing,412 rockvilla rd,null,voluntown,CT,9537,USA,False,2019-11-24T11:36:36.000+0000,2019-11-30T11:36:42.000+0000
896,Mailing,5711 nw 48 lane,null,lauderhill,FL,9537,USA,False,2019-11-24T03:23:28.000+0000,2019-11-24T15:23:33.000+0000
366,Mailing,4217 si streat,null,riverbank,CA,9537,USA,False,2019-11-24T03:10:06.000+0000,2019-11-24T03:10:12.000+0000


In [5]:
# below cells meaning:

duplicate_and_source1 (D1): table have data from OnlyDuplicateFromMaster1 and source table

In [6]:
duplicate_and_source1 = spark.sql("select ifnull(a.apid,b.apid) apid,ifnull(a.address_type,b.address_type) address_type,ifnull(a.address_line_1,b.address_line_1) address_line_1,ifnull(a.address_line_2,b.address_line_2) address_line_2,ifnull(a.city,b.city) city,ifnull(a.state,b.state) state,ifnull(a.postal_code,b.postal_code) postal_code,ifnull(a.Country_code,b.Country_code) Country_code,ifnull(a.creation_timestamp,b.creation_timestamp) creation_timestamp,ifnull(a.last_update_timestamp,b.last_update_timestamp) last_update_timestamp, ifnull(a.validated_by,b.validated_by) validated_by from OnlyDuplicateFromMaster b left join (select apid,address_type,address_line_1,address_line_2,city,state, postal_code,Country_code, validated_by ,creation_timestamp ,max(last_update_timestamp) last_update_timestamp  from table2 a group by apid,address_type,address_line_1,address_line_2,city,state, postal_code,Country_code, validated_by,creation_timestamp) a on a.apid=b.apid and a.address_line_1 =b.address_line_1 and ifnull(a.address_line_2,0) =ifnull(b.address_line_2,0) and a.city=b.city and a.state=b.state and a.postal_code=b.postal_code and a.Country_code=b.Country_code and b.last_update_timestamp <=a.last_update_timestamp")

duplicate_and_source1.createOrReplaceTempView("duplicate_and_source") 

In [7]:
display(duplicate_and_source1)

apid,address_type,address_line_1,address_line_2,city,state,postal_code,Country_code,creation_timestamp,last_update_timestamp,validated_by
896,Mailing,5711 nw 48 lane,null,lauderhill,VA,9537,USA,2019-11-24T03:23:28.000+0000,2019-11-24T15:23:33.000+0000,False
366,Mailing,4217 si streat,null,riverbank,AR,9537,USA,2019-11-24T03:10:06.000+0000,2019-11-30T03:10:12.000+0000,False
970,Mailing,412 rockvilla rd,null,voluntown,CT,9537,USA,2019-11-24T11:36:36.000+0000,2019-11-30T11:36:42.000+0000,False
896,Mailing,5711 nw 48 lane,null,lauderhill,FL,9537,USA,2019-11-24T03:23:28.000+0000,2019-11-24T15:23:33.000+0000,False
896,physical,5711 nw 48 lane,null,lauderhill,FL,9537,USA,2019-11-24T03:23:28.000+0000,2019-11-24T15:23:33.000+0000,False
366,Mailing,4217 si streat,null,riverbank,CA,9537,USA,2019-11-24T03:10:06.000+0000,2019-11-24T03:10:12.000+0000,False
366,physical,4217 si streat,null,riverbank,CA,9537,USA,2019-11-24T03:10:06.000+0000,2019-11-28T03:10:12.000+0000,False


In [8]:
%sql
select * from duplicate_and_source
union all
select * from Master_without_dublicate

apid,address_type,address_line_1,address_line_2,city,state,postal_code,Country_code,creation_timestamp,last_update_timestamp,validated_by
896,Mailing,5711 nw 48 lane,null,lauderhill,VA,9537,USA,2019-11-24T03:23:28.000+0000,2019-11-24T15:23:33.000+0000,False
366,Mailing,4217 si streat,null,riverbank,AR,9537,USA,2019-11-24T03:10:06.000+0000,2019-11-30T03:10:12.000+0000,False
970,Mailing,412 rockvilla rd,null,voluntown,CT,9537,USA,2019-11-24T11:36:36.000+0000,2019-11-30T11:36:42.000+0000,False
896,Mailing,5711 nw 48 lane,null,lauderhill,FL,9537,USA,2019-11-24T03:23:28.000+0000,2019-11-24T15:23:33.000+0000,False
896,physical,5711 nw 48 lane,null,lauderhill,FL,9537,USA,2019-11-24T03:23:28.000+0000,2019-11-24T15:23:33.000+0000,False
366,Mailing,4217 si streat,null,riverbank,CA,9537,USA,2019-11-24T03:10:06.000+0000,2019-11-24T03:10:12.000+0000,False
366,physical,4217 si streat,null,riverbank,CA,9537,USA,2019-11-24T03:10:06.000+0000,2019-11-28T03:10:12.000+0000,False
970,physical,412 rockvilla rd,null,voluntown,CT,9537,USA,2019-11-24T11:36:36.000+0000,2019-11-24T11:36:42.000+0000,False
366,physical,4217 si streat,null,riverbank,CA,9537,USA,2019-11-24T03:10:06.000+0000,2019-11-28T03:10:12.000+0000,False
896,physical,5711 nw 48 lane,null,lauderhill,FL,9537,USA,2019-11-24T03:23:28.000+0000,2019-11-24T15:23:33.000+0000,False
